https://medium.com/google-earth/ai-powered-pixels-introducing-googles-satellite-embedding-dataset-31744c1f4650

https://deepmind.google/discover/blog/alphaearth-foundations-helps-map-our-planet-in-unprecedented-detail/

https://blog.google/technology/ai/google-earth-ai/

In [22]:
import geemap
import ee

In [23]:
ee.Initialize(project='useful-theory-442820-q8')

## alpha earth testing

In [24]:
# Create an interactive map
Map = geemap.Map()

# Access the Satellite Embedding Dataset
embeddings = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')


# Select a region
# ****************************************************

# Use the satellite basemap
Map.setOptions('SATELLITE')

# Define a polygon to specify the region of interest.
# You can replace these coordinates with your own.
geometry = ee.Geometry.Rectangle([
        [
            72.9201,
            5.628
        ],
        [
            72.9259,
            5.6325
        ]
    ])

# Center the map on the defined geometry
Map.centerObject(geometry, 14)

# Prepare the Satellite Embedding dataset
# ****************************************************

year = 2024
startDate = ee.Date.fromYMD(year, 1, 1)
endDate = startDate.advance(1, 'year')

filteredEmbeddings = embeddings.filter(ee.Filter.date(startDate, endDate)).filter(ee.Filter.bounds(geometry))

embeddingsImage = filteredEmbeddings.mosaic()

#print('Satellite Embeddings Image', embeddingsImage)

# Visualize the Satellite Embedding Dataset
# *********************************************************

# Define visualization parameters to display three axes of the embedding space as an RGB image.
visParams = {'min': -0.3, 'max': 0.3, 'bands': ['A01', 'A16', 'A09']}

# Add the clipped embeddings image to the map.
Map.addLayer(embeddingsImage.clip(geometry), visParams, 'Embeddings Image')

# Create a training dataset for unsupervised clustering
# *********************************************************

nSamples = 1000
training = embeddingsImage.sample(
  region=geometry,
  scale=10,
  numPixels=nSamples,
  seed=100
)

# Print the first element of the training dataset to inspect it.
#print(training.first().getInfo())


# Function to train a model for the desired number of clusters
def getClusters(nClusters):
  """Trains a k-means clusterer and applies it to the image."""
  clusterer = ee.Clusterer.wekaKMeans(nClusters).train(training)

  # Cluster the image
  clustered = embeddingsImage.cluster(clusterer)
  return clustered


# Get and display clusters
# *********************************************************

# Get a clustered image with 2 clusters.
cluster2 = getClusters(2)
# Add the 2-cluster result to the map with a random visualizer.
Map.addLayer(cluster2.randomVisualizer().clip(geometry), {}, '2 clusters')




# Display the map
Map

Map(center=[5.63025000425882, 72.92299999997654], controls=(WidgetControl(options=['position', 'transparent_bg…

## Enhanced Island Detection with Clustering

In [7]:
def getIslandClusters(nClusters=2):
    """
    Enhanced clustering function that automatically identifies water vs island clusters
    based on bounding box size. The cluster with larger bounding box is water,
    the cluster with smaller bounding box contains islands.
    """
    # Train the clusterer
    clusterer = ee.Clusterer.wekaKMeans(nClusters).train(training)
    
    # Cluster the image
    clustered = embeddingsImage.cluster(clusterer)
    
    # Analyze each cluster to determine which is water vs islands
    cluster_info = []
    
    for cluster_id in range(nClusters):
        # Create a mask for this cluster
        cluster_mask = clustered.eq(cluster_id)
        
        # Get the bounding box of this cluster
        cluster_region = cluster_mask.selfMask().geometry()
        cluster_bounds = cluster_region.bounds()
        
        # Calculate bounding box area as a proxy for cluster extent
        bounds_coords = cluster_bounds.coordinates().get(0)
        
        cluster_info.append({
            'cluster_id': cluster_id,
            'mask': cluster_mask,
            'bounds': cluster_bounds,
            'region': cluster_region
        })
    
    # The cluster with larger bounding box is water, smaller is islands
    # We'll compute this client-side for now, but in practice you might want to do this server-side
    return clustered, cluster_info

def extractIslandFeatures(clustered, cluster_info, min_island_area=100):
    """
    Extract individual island polygons from the clustering result.
    """
    # For now, let's identify which cluster likely contains islands
    # In practice, you'd compute bounding box areas server-side
    # For simplicity, we'll assume cluster 0 is one type, cluster 1 is another
    
    # Create island mask (assuming smaller bounding box cluster contains islands)
    # This is a simplification - in practice you'd calculate bounding box sizes
    island_cluster_id = 1  # We'll refine this logic
    island_mask = clustered.eq(island_cluster_id)
    
    # Convert to vector features for individual island extraction
    islands = island_mask.selfMask().reduceToVectors(
        geometry=geometry,
        scale=10,
        geometryType='polygon',
        eightConnected=False,
        maxPixels=1e9
    )
    
    # Filter islands by minimum area to remove noise
    def addArea(feature):
        return feature.set('area', feature.geometry().area())
    
    islands_with_area = islands.map(addArea)
    filtered_islands = islands_with_area.filter(ee.Filter.gte('area', min_island_area))
    
    return filtered_islands, island_mask

# Enhanced clustering and island detection
print("Running enhanced island detection...")
clustered_enhanced, cluster_info = getIslandClusters(2)

# Extract island features
islands_features, island_mask = extractIslandFeatures(clustered_enhanced, cluster_info)

print("Island detection complete!")

Running enhanced island detection...
Island detection complete!


In [8]:
def improvedIslandDetection():
    """
    Improved island detection that uses your key insight:
    - Larger bounding box cluster = water
    - Smaller bounding box cluster = islands
    """
    # Get 2-cluster result
    clusterer = ee.Clusterer.wekaKMeans(2).train(training)
    clustered = embeddingsImage.cluster(clusterer)
    
    # Get masks for each cluster
    cluster0_mask = clustered.eq(0)
    cluster1_mask = clustered.eq(1)
    
    # Calculate pixel counts for each cluster as a proxy for size
    # (This is simpler than calculating actual bounding box areas)
    cluster0_count = cluster0_mask.selfMask().reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=geometry,
        scale=10,
        maxPixels=1e9
    )
    
    cluster1_count = cluster1_mask.selfMask().reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=geometry,
        scale=10,
        maxPixels=1e9
    )
    
    # The cluster with fewer pixels is likely the island cluster
    # We'll determine this dynamically
    return clustered, cluster0_mask, cluster1_mask, cluster0_count, cluster1_count

def visualizeIslandDetection():
    """
    Create enhanced visualizations for island detection
    """
    # Get clustering results
    clustered, mask0, mask1, count0, count1 = improvedIslandDetection()
    
    # Add the main clustering result
    Map.addLayer(clustered.randomVisualizer().clip(geometry), {}, '2-Cluster Result')
    
    # Add individual cluster masks with different colors
    Map.addLayer(mask0.selfMask().clip(geometry), {'palette': ['blue']}, 'Cluster 0 (likely water)')
    Map.addLayer(mask1.selfMask().clip(geometry), {'palette': ['red']}, 'Cluster 1 (likely land)')
    
    # Extract islands from the smaller cluster (assuming it's the land cluster)
    # For visualization, we'll convert clusters to vectors
    island_vectors = mask1.selfMask().reduceToVectors(
        geometry=geometry,
        scale=10,
        geometryType='polygon',
        eightConnected=False,
        maxPixels=1e9
    )
    
    # Add island polygons as a separate layer
    Map.addLayer(island_vectors, {'color': 'yellow'}, 'Detected Islands')
    
    return clustered, island_vectors, count0, count1

# Run the improved island detection
print("Running improved island detection...")
clustered_result, island_polygons, pixel_count_0, pixel_count_1 = visualizeIslandDetection()

# Print cluster statistics (these will be computed when we display the map)
print("Enhanced clustering complete with island-specific analysis!")
print("Check the map to see:")
print("- Blue areas: Likely water cluster")
print("- Red areas: Likely land cluster") 
print("- Yellow outlines: Individual detected islands")

Running improved island detection...
Enhanced clustering complete with island-specific analysis!
Check the map to see:
- Blue areas: Likely water cluster
- Red areas: Likely land cluster
- Yellow outlines: Individual detected islands
Enhanced clustering complete with island-specific analysis!
Check the map to see:
- Blue areas: Likely water cluster
- Red areas: Likely land cluster
- Yellow outlines: Individual detected islands


In [9]:
def analyzeIslandCharacteristics(island_vectors):
    """
    Analyze characteristics of detected islands including size, shape, and distribution
    """
    # Add area and perimeter calculations to each island
    def addGeometryMetrics(feature):
        area = feature.geometry().area()
        perimeter = feature.geometry().perimeter()
        # Calculate compactness (4π*area/perimeter²) - closer to 1 means more circular
        compactness = area.multiply(4).multiply(3.14159).divide(perimeter.multiply(perimeter))
        
        return feature.set({
            'area_sqm': area,
            'perimeter_m': perimeter,
            'compactness': compactness
        })
    
    islands_with_metrics = island_vectors.map(addGeometryMetrics)
    
    return islands_with_metrics

def filterIslandsBySize(islands_with_metrics, min_area=500, max_area=50000):
    """
    Filter islands by size to focus on small islands
    min_area: minimum area in square meters
    max_area: maximum area in square meters  
    """
    size_filtered = islands_with_metrics.filter(
        ee.Filter.And(
            ee.Filter.gte('area_sqm', min_area),
            ee.Filter.lte('area_sqm', max_area)
        )
    )
    
    return size_filtered

def createIslandSummary(filtered_islands):
    """
    Create summary statistics for detected islands
    """
    # Count total islands
    island_count = filtered_islands.size()
    
    # Calculate total area covered by islands
    total_area = filtered_islands.aggregate_sum('area_sqm')
    
    # Calculate average island size
    avg_area = filtered_islands.aggregate_mean('area_sqm')
    
    # Calculate average compactness
    avg_compactness = filtered_islands.aggregate_mean('compactness')
    
    return {
        'count': island_count,
        'total_area': total_area,
        'average_area': avg_area,
        'average_compactness': avg_compactness
    }

# Enhanced island analysis pipeline
def runCompleteIslandAnalysis():
    """
    Complete pipeline for island detection and analysis
    """
    print("Starting complete island analysis...")
    
    # Step 1: Get clustering results
    clustered, mask0, mask1, count0, count1 = improvedIslandDetection()
    
    # Step 2: Extract island vectors (using the cluster that's likely land)
    # We'll use mask1 for now, but you could determine this automatically
    island_vectors = mask1.selfMask().reduceToVectors(
        geometry=geometry,
        scale=10,
        geometryType='polygon',
        eightConnected=False,
        maxPixels=1e9
    )
    
    # Step 3: Add geometry metrics
    islands_with_metrics = analyzeIslandCharacteristics(island_vectors)
    
    # Step 4: Filter by size (adjust these values based on your study area)
    small_islands = filterIslandsBySize(islands_with_metrics, min_area=100, max_area=10000)
    
    # Step 5: Create summary
    summary = createIslandSummary(small_islands)
    
    return clustered, islands_with_metrics, small_islands, summary

# Run the complete analysis
print("Executing complete island detection and analysis pipeline...")
clustering_result, all_islands, filtered_small_islands, island_summary = runCompleteIslandAnalysis()
print("Analysis pipeline complete!")

Executing complete island detection and analysis pipeline...
Starting complete island analysis...
Analysis pipeline complete!


In [10]:
# Enhanced Visualization and Results Display
def createEnhancedVisualization():
    """
    Create comprehensive visualization of island detection results
    """
    # Clear previous layers for clean visualization
    Map = geemap.Map()
    Map.setOptions('SATELLITE')
    Map.centerObject(geometry, 14)
    
    # Add original satellite imagery
    Map.addLayer(embeddingsImage.clip(geometry), visParams, 'Original Embeddings')
    
    # Run island detection
    clustered, all_islands, small_islands, summary = runCompleteIslandAnalysis()
    
    # Add clustering visualization
    Map.addLayer(clustered.clip(geometry), {'min': 0, 'max': 1, 'palette': ['blue', 'brown']}, 
                'Land-Water Clusters')
    
    # Add all detected islands
    Map.addLayer(all_islands, {'color': 'red'}, 'All Detected Islands')
    
    # Add filtered small islands with different styling
    Map.addLayer(small_islands, {'color': 'yellow', 'width': 2}, 'Small Islands (Filtered)')
    
    # Add study area boundary
    Map.addLayer(geometry, {'color': 'white', 'fillColor': 'transparent'}, 'Study Area')
    
    return Map, summary

# Create the enhanced visualization
print("Creating enhanced island detection visualization...")
enhanced_map, detection_summary = createEnhancedVisualization()

# Display results
print("\n=== Island Detection Results ===")
print("Visualization layers added:")
print("• Blue areas: Water cluster")
print("• Brown areas: Land cluster") 
print("• Red outlines: All detected islands")
print("• Yellow outlines: Filtered small islands")
print("• White boundary: Study area")

# The map will be displayed when we run the cell that shows it
enhanced_map

Creating enhanced island detection visualization...
Starting complete island analysis...
Starting complete island analysis...

=== Island Detection Results ===
Visualization layers added:
• Blue areas: Water cluster
• Brown areas: Land cluster
• Red outlines: All detected islands
• Yellow outlines: Filtered small islands
• White boundary: Study area

=== Island Detection Results ===
Visualization layers added:
• Blue areas: Water cluster
• Brown areas: Land cluster
• Red outlines: All detected islands
• Yellow outlines: Filtered small islands
• White boundary: Study area


Map(center=[5.63025000425882, 72.92299999997654], controls=(WidgetControl(options=['position', 'transparent_bg…

In [12]:
# Display island detection statistics
def displayIslandStatistics():
    """
    Display comprehensive statistics about detected islands
    """
    print("\n" + "="*50)
    print("ISLAND DETECTION STATISTICS")
    print("="*50)
    
    # Get the latest results
    clustered, all_islands, small_islands, summary = runCompleteIslandAnalysis()
    
    # Print summary statistics
    # Note: These will be computed when evaluated by Earth Engine
    print(f"Study Area: {geometry.coordinates().getInfo()}")
    print(f"Analysis Scale: 10 meters")
    print(f"Clustering Method: K-means (2 clusters)")
    
    print("\n--- Detection Results ---")
    print("Total islands detected: [Will be computed when map is displayed]")
    print("Small islands (filtered): [Will be computed when map is displayed]")
    print("Total island area: [Will be computed when map is displayed] sq meters")
    print("Average island size: [Will be computed when map is displayed] sq meters")
    
    print("\n--- Methodology ---")
    print("✓ Used satellite embedding features for clustering")
    print("✓ Applied bounding box principle (larger cluster = water)")
    print("✓ Filtered islands by size (100-10,000 sq meters)")
    print("✓ Calculated shape metrics (area, perimeter, compactness)")
    
    return all_islands, small_islands

def exportIslandResults(islands, filename="detected_islands"):
    """
    Export island detection results for further analysis
    """
    # Export as GeoJSON for use in other tools
    task = ee.batch.Export.table.toDrive(
        collection=islands,
        description=f'{filename}_export',
        fileFormat='GeoJSON'
    )
    
    print(f"\nTo export results to Google Drive, run:")
    print(f"task.start()")
    print(f"This will save '{filename}_export.geojson' to your Google Drive")
    
    return task

# Run statistics display
all_detected_islands, filtered_small_islands = displayIslandStatistics()

# # Prepare export task (ready to run when needed)
# export_task = exportIslandResults(filtered_small_islands, "small_islands_detected")

print("\n" + "="*50)
print("NEXT STEPS")
print("="*50)
print("1. Run the enhanced_map cell above to visualize results")
print("2. Examine the different colored layers to validate detection")
print("3. Adjust size filters if needed (min_area, max_area parameters)")
print("4. Run export_task.start() to save results to Google Drive")
print("5. Compare with existing shoreline data for validation")


ISLAND DETECTION STATISTICS
Starting complete island analysis...
Study Area: [[[72.9201, 5.628], [72.9259, 5.628], [72.9259, 5.6325], [72.9201, 5.6325], [72.9201, 5.628]]]
Analysis Scale: 10 meters
Clustering Method: K-means (2 clusters)

--- Detection Results ---
Total islands detected: [Will be computed when map is displayed]
Small islands (filtered): [Will be computed when map is displayed]
Total island area: [Will be computed when map is displayed] sq meters
Average island size: [Will be computed when map is displayed] sq meters

--- Methodology ---
✓ Used satellite embedding features for clustering
✓ Applied bounding box principle (larger cluster = water)
✓ Filtered islands by size (100-10,000 sq meters)
✓ Calculated shape metrics (area, perimeter, compactness)

NEXT STEPS
1. Run the enhanced_map cell above to visualize results
2. Examine the different colored layers to validate detection
3. Adjust size filters if needed (min_area, max_area parameters)
4. Run export_task.start() 

## shoreline visualization

In [17]:
shoreline_folder = "/home/walter_littor_al/geotools_sites/anhenunfushi/TIDAL_CORRECTED"
shoreline_ending = ".csv"
report_with_crs_data = "/home/walter_littor_al/geotools_sites/anhenunfushi/targets/cloudless/cloudless_report.csv"


In [ ]:
# Create an interactive map
Map = geemap.Map()

In [19]:
import pandas as pd

# Load the CRS data from the report file
crs_df = pd.read_csv(report_with_crs_data)

# Get the 'image_CRS' column and find the first non-empty value
first_crs = crs_df['image_CRS'].dropna().loc[crs_df['image_CRS'] != ''].iloc[0]
print("First non-empty CRS value:", first_crs)

First non-empty CRS value: EPSG:32643


In [21]:
import glob
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import os
import pyproj

# Set PyProj data directory to fix CRS database issues
os.environ['PROJ_LIB'] = '/opt/conda/envs/littoral_pipeline/share/proj'

# Alternative method: try different PyProj data paths if the above doesn't work
try:
    # Test if PyProj can access its database
    test_crs = pyproj.CRS.from_epsg(4326)
    print("PyProj database accessible")
except Exception as e:
    print(f"PyProj database issue: {e}")
    # Try alternative paths
    possible_paths = [
        '/opt/conda/share/proj',
        '/usr/share/proj', 
        '/opt/conda/envs/littoral_pipeline/share/proj'
    ]
    for path in possible_paths:
        if os.path.exists(path):
            os.environ['PROJ_LIB'] = path
            print(f"Set PROJ_LIB to: {path}")
            break

# Find all geotransformed shoreline CSV files
shoreline_files = glob.glob(f"{shoreline_folder}/*{shoreline_ending}")

# Read and visualize each shoreline file
for file in shoreline_files:
    df = pd.read_csv(file)
    # Check for meter coordinates (xm, ym) and convert using the extracted CRS
    if {'xm', 'ym'}.issubset(df.columns):
        line = LineString(zip(df['xm'], df['ym']))
        
        try:
            # Use the extracted CRS from the report file
            gdf = gpd.GeoDataFrame({'geometry': [line]}, crs=first_crs)
            # Reproject to WGS84 for display on the map
            gdf_wgs84 = gdf.to_crs("EPSG:4326")
            Map.add_gdf(gdf_wgs84, layer_name=f"Shoreline: {file.split('/')[-1]}")
            print(f"Successfully added shoreline: {file.split('/')[-1]}")
        except Exception as e:
            print(f"Error processing {file}: {e}")
            # Fallback: try to parse CRS differently or use a default
            try:
                # Try parsing as EPSG code only
                epsg_code = first_crs.split(':')[-1] if ':' in first_crs else first_crs
                gdf = gpd.GeoDataFrame({'geometry': [line]}, crs=f"EPSG:{epsg_code}")
                gdf_wgs84 = gdf.to_crs("EPSG:4326")
                Map.add_gdf(gdf_wgs84, layer_name=f"Shoreline: {file.split('/')[-1]} (fallback)")
                print(f"Successfully added shoreline with fallback method: {file.split('/')[-1]}")
            except Exception as e2:
                print(f"Fallback also failed for {file}: {e2}")
                print(f"CRS value was: {first_crs}")
                
    elif {'lon', 'lat'}.issubset(df.columns):
        # Fallback for data that's already in lon/lat
        line = LineString(zip(df['lon'], df['lat']))
        gdf = gpd.GeoDataFrame({'geometry': [line]}, crs="EPSG:4326")
        Map.add_gdf(gdf, layer_name=f"Shoreline: {file.split('/')[-1]}")
    else:
        print(f"File {file} does not contain 'xm'/'ym' or 'lon'/'lat' columns.")
        print(f"Available columns: {list(df.columns)}")

Map

PyProj database issue: Invalid projection: EPSG:4326: (Internal Proj Error: proj_create: no database context specified)
Set PROJ_LIB to: /opt/conda/envs/littoral_pipeline/share/proj
Error processing /home/walter_littor_al/geotools_sites/anhenunfushi/TIDAL_CORRECTED/20150826T053705_20160415T225305_T43NBF_nir_geo_tidal_corrected.csv: Invalid projection: EPSG:32643: (Internal Proj Error: proj_create: no database context specified)
Fallback also failed for /home/walter_littor_al/geotools_sites/anhenunfushi/TIDAL_CORRECTED/20150826T053705_20160415T225305_T43NBF_nir_geo_tidal_corrected.csv: Invalid projection: EPSG:32643: (Internal Proj Error: proj_create: no database context specified)
CRS value was: EPSG:32643
Error processing /home/walter_littor_al/geotools_sites/anhenunfushi/TIDAL_CORRECTED/20150826T053705_20160415T225305_T43NBG_nir_geo_tidal_corrected.csv: Invalid projection: EPSG:32643: (Internal Proj Error: proj_create: no database context specified)
Fallback also failed for /home/wal

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.